In [1]:
# noexport

import os
os.system('export_notebook identify_domain.ipynb')

0

In [2]:
from tmilib import *

In [3]:
#user = get_test_users()[0]
#print user

In [4]:
#tab_focus_times = get_tab_focus_times_for_user(user)
#print tab_focus_times[0].keys()

In [5]:
#print active_second_to_domain_id.keys()[:10]

In [6]:
@memoized
def get_seen_urls_and_domains_real_and_history():
  seen_urls_history = Counter()
  seen_domains_history = Counter()
  for user in get_test_users():
    active_seconds_set = set(get_active_insession_seconds_for_user(user))
    ordered_visits = get_history_ordered_visits_corrected_for_user(user)
    ordered_visits = exclude_bad_visits(ordered_visits)
    for visit in ordered_visits:
      url = visit['url']
      visit_time = visit['visitTime']
      if visit_time not in active_seconds_set:
        continue
      #if 'chrome://' in url:
      #  chrome_urls[url] += 1
      seen_urls_history[url] += 1
      seen_domains_history[url_to_domain(url)] += 1

  seen_urls_real = Counter()
  seen_domains_real = Counter()
  for user in get_test_users():
    active_seconds_set = set(get_active_insession_seconds_for_user(user))
    for time,url in get_active_url_at_time_for_user(user).viewitems():
      if time not in active_seconds_set:
        continue
      seen_urls_real[url] += 1
      seen_domains_real[url_to_domain(url)] += 1
      #if url.startswith('chrome://'):
      #  chrome_urls[url] += 1
  
  seen_domains_only_real = {k:v for k,v in seen_domains_real.viewitems() if k not in seen_domains_history}
  #print_counter(seen_domains_only_real)

  seen_domains_only_history = {k:v for k,v in seen_domains_history.viewitems() if k not in seen_domains_real}
  #print_counter(seen_domains_only_history)
  return {
    'seen_urls_history': seen_urls_history,
    'seen_domains_history': seen_domains_history,
    'seen_urls_real': seen_urls_real,
    'seen_domains_real': seen_domains_real,
    'seen_domains_only_real': seen_domains_only_real,
    'seen_domains_only_history': seen_domains_only_history,
  }

In [7]:
#seen_domains_only_real = get_seen_urls_and_domains_real_and_history()['seen_domains_only_real']

In [8]:
print_counter(get_seen_urls_and_domains_real_and_history()['seen_domains_only_real'])

In [9]:
print_counter(get_seen_urls_and_domains_real_and_history()['seen_domains_only_history'])

In [10]:
@memoized
def get_recently_seen_domain_stats_for_user(user):
  ordered_visits = get_history_ordered_visits_corrected_for_user(user)
  ordered_visits = exclude_bad_visits(ordered_visits)
  active_domain_at_time = get_active_domain_at_time_for_user(user)
  active_seconds_set = set(get_active_insession_seconds_for_user(user))
  #active_second_to_domain_id = get_active_second_to_domain_id_for_user(user)
  active_second_to_domain_id = {int(k):v for k,v in get_active_second_to_domain_id_for_user(user).viewitems()}
  recently_seen_domain_ids = [-1]*100
  seen_domain_ids_set = set()
  stats = Counter()
  nomatch_domains = Counter()
  tabbed_back_domains_first = Counter()
  tabbed_back_domains_second = Counter()
  tabbed_back_domains_all = Counter()
  distractors_list = [domain_to_id(x) for x in ['www.mturk.com', 'apps.facebook.com', 'www.facebook.com', 'www.reddit.com', 'www.youtube.com']]
  distractors = set(distractors_list)
  most_recent_distractor = distractors_list[0]
  for idx,visit in enumerate(ordered_visits):
    if idx+1 >= len(ordered_visits):
      break
    next_visit = ordered_visits[idx+1]
    cur_domain = url_to_domain(visit['url'])
    cur_domain_id = domain_to_id(cur_domain)
    if cur_domain_id in distractors:
      most_recent_distractor = cur_domain_id
    if cur_domain_id != recently_seen_domain_ids[-1]:
      if cur_domain_id in seen_domain_ids_set:
        recently_seen_domain_ids.remove(cur_domain_id)
      seen_domain_ids_set.add(cur_domain_id)
      recently_seen_domain_ids.append(cur_domain_id)
    next_domain = url_to_domain(next_visit['url'])
    next_domain_id = domain_to_id(next_domain)
    cur_time_sec = int(round(visit['visitTime'] / 1000.0))
    next_time_sec = int(round(next_visit['visitTime'] / 1000.0))
    
    if cur_time_sec > next_time_sec:
      continue
    
    for time_sec in xrange(cur_time_sec, next_time_sec+1):
      if time_sec not in active_seconds_set:
        continue
      ref_domain_id = active_second_to_domain_id[time_sec]
      stats['total'] += 1
      if most_recent_distractor == ref_domain_id:
        stats['most_recent_distractor_total'] += 1
        if cur_domain_id == ref_domain_id:
          stats['most_recent_distractor_curdomain'] += 1
        elif cur_domain_id == next_domain_id:
          stats['most_recent_distractor_nextdomain'] += 1
        else:
          stats['most_recent_distractor_some_prev_domain'] += 1
      if cur_domain_id == ref_domain_id:
        if next_domain_id == cur_domain_id:
          stats['first and next equal and correct'] += 1
        else:
          stats['first correct only'] += 1
        continue
      if next_domain_id == ref_domain_id:
        stats['next correct only'] += 1
        continue
      stats['both incorrect'] += 1
      found_match = False
      ref_domain = id_to_domain(ref_domain_id)
      for i in range(1,101):
        if recently_seen_domain_ids[-1-i] == ref_domain_id:
          stats['nth previous correct ' + str(abs(i))] += 1
          stats['some previous among past 100 correct'] += 1
          found_match = True
          tabbed_back_domains_all[ref_domain] += 1
          if i == 1:
            tabbed_back_domains_first[ref_domain] += 1
          if i == 2:
            tabbed_back_domains_second[ref_domain] += 1
          break
      if not found_match:
        ref_domain = id_to_domain(ref_domain_id)
        if ref_domain == 'newtab':
          stats['newtab'] += 1
          continue
        stats['no match found'] += 1
        nomatch_domains[id_to_domain(ref_domain_id)] += 1
      '''
      if cur_domain_id == ref_domain_id:
        if next_domain_id == cur_domain_id:
          stats['first and next equal and correct'] += 1
          continue
        else:
          stats['first correct only'] += 1
          continue
      else:
        if next_domain_id == cur_domain_id:
          stats['both incorrect'] += 1
          found_match = False
          for i in range(1,101):
            if recently_seen_domain_ids[-1-i] == ref_domain_id:
              stats['nth previous correct ' + str(abs(i))] += 1
              stats['some previous among past 100 correct'] += 1
              found_match = True
              break
          if not found_match:
            ref_domain = id_to_domain(ref_domain_id)
            if ref_domain == 'newtab':
              stats['newtab'] += 1
              continue
            stats['no match found'] += 1
            nomatch_domains[id_to_domain(ref_domain_id)] += 1
          continue
        if next_domain_id == ref_domain_id:
          stats['next correct only'] += 1
          continue
      '''
  return {
    'stats': stats,
    'nomatch_domains': nomatch_domains,
    'tabbed_back_domains_all': tabbed_back_domains_all,
    'tabbed_back_domains_first': tabbed_back_domains_first,
    'tabbed_back_domains_second': tabbed_back_domains_second,
  }

In [11]:
total_nomatch_domains = Counter()

for user in get_test_users():
  for k,v in get_recently_seen_domain_stats_for_user(user)['nomatch_domains'].viewitems():
    total_nomatch_domains[k] += v


In [12]:
#print total_nomatch_domains
print_counter(total_nomatch_domains)

bookmarks 8379
tmi.stanford.edu 5134
www.mturk.com 4215
www.nwanime.com 3841
 2640
www.youtube.com 2366
www.facebook.com 2341
www.amazon.com 2017
klbibkeccnjlkjkiokjodocebajanakg 1750
downloads 1714
history 1680
settings 1583
www.googleadservices.com 1314
www.google.com 1105
ww2.unipark.de 1038
survey18.toluna.com 995
extensions 961
survey.theacsi.org 954
plinga.com 947
templates.textbroker.com 918
adclick.g.doubleclick.net 883
gsu.qualtrics.com 803
mail.google.com 800
survey.az1.qualtrics.com 780
ups.surveyrouter.com 768
www.heatherridley.com 712
survey13.toluna.com 687
www.clearvoicesurveys.com 685
features.texasmonthly.com 633
www.iheart.com 596
www.reddit.com 595
usccmcis.az1.qualtrics.com 586
www.surveymonkey.com 530
vassarpsych.az1.qualtrics.com 511
armorgames.com 491
www.gamestop.com 486
usccollege.qualtrics.com 484
uky.az1.qualtrics.com 472
uwmadison.co1.qualtrics.com 465
drive.google.com 450
hbs.qualtrics.com 429
www.pandaresearch.com 423
gngocbkfmikdgphklgmmehbjjlfgdemm 411
w

In [13]:
total_tabbed_back_domains_all = Counter()

for user in get_test_users():
  for k,v in get_recently_seen_domain_stats_for_user(user)['tabbed_back_domains_all'].viewitems():
    total_tabbed_back_domains_all[k] += v

In [14]:
#[domain_to_id(x) for x in ['www.mturk.com', 'apps.facebook.com', 'www.facebook.com', 'www.reddit.com', 'www.youtube.com']]

In [15]:
#sumkeys(total_tabbed_back_domains_all, 'www.mturk.com', 'apps.facebook.com', 'www.facebook.com', 'www.reddit.com', 'www.youtube.com')

In [16]:
print sum(total_tabbed_back_domains_all.values())

2515898


In [17]:
#728458.0/2382221

In [18]:
print_counter(total_tabbed_back_domains_all)

www.mturk.com 203806
www.facebook.com 192141
www.youtube.com 97316
www.reddit.com 83404
forums.somethingawful.com 81583
mail.google.com 58793
www.google.com 51957
allmyvideos.net 31371
templates.textbroker.com 28956
www.tumblr.com 28808
www.amazon.com 28626
hitgrabber.net 24706
apps.facebook.com 21464
vidbull.com 21069
www.netflix.com 19499
docs.google.com 17526
www.amzreviewtrader.com 16451
wharton.qualtrics.com 15865
www.arconaitv.me 15677
www.speechpad.com 14804
upenn.co1.qualtrics.com 14668
kellogg.qualtrics.com 14310
www.swagbucks.com 13185
us-mg5.mail.yahoo.com 12925
www.allmyvideos.net 12324
mprlab327.webfactional.com 11675
drive.google.com 11542
intern.textbroker.com 11485
armorgames.com 11310
mturkforum.com 10963
plinga.com 10764
forums.animez.to 10633
mail.aol.com 10632
yalesurvey.qualtrics.com 10279
www.addictinggames.com 10234
tweetdeck.twitter.com 9909
iu.co1.qualtrics.com 9237
washington.co1.qualtrics.com 8963
myanimelist.net 8556
stanfordgsb.qualtrics.com 8555
www.crichd

In [19]:
#print_counter(total_tabbed_back_domains_first)

In [20]:
'''
total_active_seconds = 0

for user in get_test_users():
  total_active_seconds += len(get_active_seconds_for_user(user))
  #for span in get_active_seconds_for_user(user)
print total_active_seconds
'''

'\ntotal_active_seconds = 0\n\nfor user in get_test_users():\n  total_active_seconds += len(get_active_seconds_for_user(user))\n  #for span in get_active_seconds_for_user(user)\nprint total_active_seconds\n'

In [21]:
#for user in get_test_users():
#  print user
#  print (get_recently_seen_domain_stats_for_user(user)['stats'])

In [22]:
#total_stats = Counter({'total': 544544, 'first and next equal and correct': 351081, 'first correct only': 88522, 'both incorrect': 51663, 'some previous among past 20 correct': 41231, 'nth previous correct 1': 31202, 'next correct only': 23569, 'no match found': 10432, 'nth previous correct 2': 3311, 'nth previous correct 3': 1635, 'nth previous correct 4': 905, 'nth previous correct 5': 862, 'nth previous correct 6': 545, 'nth previous correct 7': 412, 'nth previous correct 8': 357, 'nth previous correct 9': 269, 'nth previous correct 10': 259, 'nth previous correct 13': 234, 'nth previous correct 11': 229, 'nth previous correct 12': 190, 'nth previous correct 15': 183, 'nth previous correct 14': 140, 'nth previous correct 17': 139, 'nth previous correct 20': 95, 'nth previous correct 16': 90, 'nth previous correct 19': 88, 'nth previous correct 18': 86})

total_stats = Counter()

for user in get_test_users():
  for k,v in get_recently_seen_domain_stats_for_user(user)['stats'].viewitems():
    total_stats[k] += v
#total_stats = Counter({'total': 544544, 'first and next equal and correct': 351081, 'first correct only': 88522, 'both incorrect': 51663, 'some previous among past 20 correct': 41136, 'nth previous correct 2': 31202, 'next correct only': 23569, 'no match found': 10527, 'nth previous correct 3': 3311, 'nth previous correct 4': 1635, 'nth previous correct 5': 905, 'nth previous correct 6': 862, 'nth previous correct 7': 545, 'nth previous correct 8': 412, 'nth previous correct 9': 357, 'nth previous correct 10': 269, 'nth previous correct 11': 259, 'nth previous correct 14': 234, 'nth previous correct 12': 229, 'nth previous correct 13': 190, 'nth previous correct 16': 183, 'nth previous correct 15': 140, 'nth previous correct 18': 139, 'nth previous correct 17': 90, 'nth previous correct 20': 88, 'nth previous correct 19': 86})

In [23]:
print_counter(total_stats)

total 14806666
first and next equal and correct 6588587
most_recent_distractor_total 4978110
first correct only 4430669
most_recent_distractor_curdomain 4226119
both incorrect 2692997
some previous among past 100 correct 2515898
nth previous correct 1 1096993
next correct only 1094413
most_recent_distractor_some_prev_domain 621347
nth previous correct 2 465806
nth previous correct 3 264611
nth previous correct 4 160500
most_recent_distractor_nextdomain 130644
nth previous correct 5 94361
no match found 89836
newtab 87263
nth previous correct 6 80847
nth previous correct 7 54738
nth previous correct 8 44045
nth previous correct 9 32883
nth previous correct 10 32032
nth previous correct 11 21896
nth previous correct 13 18950
nth previous correct 12 18078
nth previous correct 15 11197
nth previous correct 18 10402
nth previous correct 17 9417
nth previous correct 14 9087
nth previous correct 16 8940
nth previous correct 19 6453
nth previous correct 21 6121
nth previous correct 20 5679
nth

In [24]:
def sumkeys(d, *args):
  return sum(d.get(x, 0.0) for x in args)

In [25]:
norm = {k:float(v)/total_stats['total'] for k,v in total_stats.viewitems()}
print 'select prev gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only')
print 'prev or next gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only')
print 'prev or next or newtab gets answer correct', sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only', 'newtab')
for i in range(1, 101):
  sumprev = sum([norm.get('nth previous correct '+str(x),0.0) for x in range(i+1)])
  print 'prev or next or past ' + str(i), sumkeys(norm, 'first and next equal and correct', 'first correct only', 'next correct only', 'newtab')+sumprev

select prev gets answer correct 0.74420912851
prev or next gets answer correct 0.818122661779
prev or next or newtab gets answer correct 0.824016155967
prev or next or past 1 0.898103935079
prev or next or past 2 0.92956314406
prev or next or past 3 0.947434216454
prev or next or past 4 0.958273928783
prev or next or past 5 0.96464680165
prev or next or past 6 0.970106977492
prev or next or past 7 0.973803825925
prev or next or past 8 0.97677849963
prev or next or past 9 0.978999323683
prev or next or past 10 0.981162673623
prev or next or past 11 0.982641467026
prev or next or past 12 0.983862403596
prev or next or past 13 0.985142232559
prev or next or past 14 0.985755942627
prev or next or past 15 0.986512156079
prev or next or past 16 0.987115938186
prev or next or past 17 0.987751935513
prev or next or past 18 0.988454456932
prev or next or past 19 0.988890274151
prev or next or past 20 0.989273817617
prev or next or past 21 0.989687212503
prev or next or past 22 0.990013754616
pr

In [26]:
#print norm['most_recent_distractor_total']
#print norm['most_recent_distractor_curdomain']
#print norm['most_recent_distractor_nextdomain']
#print norm['most_recent_distractor_some_prev_domain']